<a href="https://colab.research.google.com/github/ilhaanrahim/Match-Making-Matrimony/blob/main/Model_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers numpy scikit-learn


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install transformers torch spacy

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
def get_user_preferences():
    print("Welcome to the Improved Profile Matcher!")

    print("What is your religion? (e.g., Islam, Christianity, Hinduism, etc.)")
    religion = input().strip().lower()


    print("What are you looking for in a partner? (Please describe in a few sentences)")
    partner_preferences = input().strip()

    print("Please enter your full 'About Me' description:")
    full_profile = input().strip()

    return religion, partner_preferences, full_profile

def is_religion_match(user_religion, profile_religion):
    religion_mappings = {
        "islam": ["muslim", "islam"],
        "christianity": ["christian", "christianity"],
        "hinduism": ["hindu", "hinduism"],
        "buddhism": ["buddhist", "buddhism"],
        "sikhism": ["sikh", "sikhism"],
        "judaism": ["jewish", "judaism"]
    }

    if user_religion in religion_mappings:
        return profile_religion in religion_mappings[user_religion]
    else:
        return user_religion == profile_religion

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [4]:

from spacy.matcher import Matcher

def extract_profile_info(text):
    doc = nlp(text)
    info = {
        "religion": None,
        "interests": [],
        "preferences": [],
        "exclusions": []
    }

    # Define patterns for matcher
    matcher = Matcher(nlp.vocab)
    matcher.add("RELIGION", [[{"LOWER": "muslim"}], [{"LOWER": "christian"}], [{"LOWER": "hindu"}], [{"LOWER": "buddhist"}], [{"LOWER": "sikh"}], [{"LOWER": "jewish"}], [{"LOWER": "atheist"}]])
    matcher.add("EXCLUSION", [[{"LOWER": "not"}, {"LOWER": "looking"}, {"LOWER": "for"}], [{"LOWER": "don't"}, {"LOWER": "want"}], [{"LOWER": "avoid"}]])

    matches = matcher(doc)

    for match_id, start, end in matches:
        span = doc[start:end]
        if nlp.vocab.strings[match_id] == "RELIGION":
            if any(token.dep_ == "neg" for token in span.sent):
                info["exclusions"].append(span.text.lower())
            else:
                info["religion"] = span.text.lower()
        elif nlp.vocab.strings[match_id] == "EXCLUSION":
            info["exclusions"].extend([token.text.lower() for token in span.sent if token.pos_ in ["NOUN", "PROPN", "ADJ"]])

    for token in doc:
        if token.pos_ in ["NOUN", "VERB", "ADJ"] and token.dep_ not in ["aux", "auxpass"]:
            if any(excl in token.sent.text.lower() for excl in ["not", "don't", "avoid"]):
                continue
            elif any(pref in token.sent.text.lower() for pref in ["want", "looking for", "prefer", "must", "only"]):
                info["preferences"].append(token.lemma_.lower())
            else:
                info["interests"].append(token.lemma_.lower())

    return info

In [5]:
def calculate_match_score(user_religion, user_preferences, user_profile, profile_info, profile_embedding):
    # Check for religion match
    if not is_religion_match(user_religion, profile_info['religion']):
        return 0

    # Calculate semantic similarity for preferences
    user_pref_embedding = generate_embedding(user_preferences)
    profile_pref_embedding = generate_embedding(" ".join(profile_info['preferences']))
    pref_similarity = cosine_similarity([user_pref_embedding], [profile_pref_embedding])[0][0]

    # Calculate overall profile similarity
    user_profile_embedding = generate_embedding(user_profile)
    profile_similarity = cosine_similarity([user_profile_embedding], [profile_embedding])[0][0]

    # Combine scores (you can adjust weights as needed)
    score = 0.4 * pref_similarity + 0.6 * profile_similarity

    return score

In [6]:
about_me_data = [
   "I love hiking and exploring nature. I'm looking for someone who enjoys outdoor activities too. I am looking for practising hindu",
"I'm passionate about cooking and trying new cuisines. I'm seeking a food enthusiast to share culinary adventures with.",
"As a tech geek working in AI, I'm interested in someone who can discuss the latest innovations with me. I am not looking for a muslim",
"I'm an avid reader and writer, and I hope to find a partner who appreciates literature and intellectual conversations.",
"I'm a fitness freak and yoga instructor, searching for a health-conscious individual to share an active lifestyle with.",
"As a travel junkie always planning the next adventure, I want to meet someone with wanderlust.",
"I'm a music lover and amateur guitarist, looking for someone to jam with and attend concerts together.",
"As an animal lover and volunteer at a local shelter, I'm hoping to find a fellow pet enthusiast.",
"I'm an art enthusiast and painter, seeking a creative soul to explore galleries and museums with.",
"I'm an environmentalist passionate about sustainability, and I want to connect with someone who cares about the planet. I am a practising muslim",
    "I'm a Muslim seeking a partner who shares my faith. I enjoy outdoor activities and cooking.",
    "Looking for a non-smoker who loves travel and adventure. No party animals please.",
    "Christian seeking same. I'm passionate about community service and my faith.",
    "Atheist interested in science and philosophy. Not looking for religious partners.",
    "Vegetarian animal lover. Seeking someone who doesn't eat meat and enjoys nature.",
    "Muslim professional interested in tech and AI. I pray five times a day and want a partner who does the same.",
    "Hindu looking for a life partner. I'm into yoga, meditation, and Indian classical music.",
    "Jewish software engineer. Shabbat is important to me. Looking for someone who shares these values.",
    "Buddhist artist seeking a spiritually-minded partner. I meditate daily and am involved in my local sangha.",
    "Sikh doctor looking for a partner who respects Sikh traditions. I'm passionate about seva and social justice."
]
profile_infos = [extract_profile_info(text) for text in about_me_data]
profile_embeddings = [generate_embedding(text) for text in about_me_data]

def find_matches(user_religion, user_preferences, user_profile, profile_embeddings, profile_infos, top_n=3):
    user_profile_embedding = generate_embedding(user_profile)
    scores = [calculate_match_score(user_religion, user_preferences, user_profile, p_info, p_embedding)
              for p_info, p_embedding in zip(profile_infos, profile_embeddings)]
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
    return top_indices, [scores[i] for i in top_indices]

In [7]:
def match_profiles():
    user_religion, user_preferences, user_profile = get_user_preferences()

    matched_indices, scores = find_matches(user_religion, user_preferences, user_profile, profile_embeddings, profile_infos)

    print("\nYour profile information:")
    print(f"Religion: {user_religion}")
    print(f"Partner Preferences: {user_preferences}")
    print(f"Full Profile: {user_profile}")

    print("\nTop matches for you:")
    for i, (index, score) in enumerate(zip(matched_indices, scores), 1):
        if score > 0:  # Only show non-zero scores
            print(f"{i}. Match (Score: {score:.2f}):")
            print(f"   {about_me_data[index]}")
            print(f"   Religion: {profile_infos[index]['religion']}")
            print(f"   Interests: {', '.join(profile_infos[index]['interests'])}")
            print(f"   Preferences: {', '.join(profile_infos[index]['preferences'])}")
            print(f"   Exclusions: {', '.join(profile_infos[index]['exclusions'])}\n")

# Run the matcher
match_profiles()

Welcome to the Improved Profile Matcher!
What is your religion? (e.g., Islam, Christianity, Hinduism, etc.)
Christianity
What are you looking for in a partner? (Please describe in a few sentences)
i am looking for a hindu girl who outdoor activities and cooking and a software engineer 
Please enter your full 'About Me' description:
Hey there! I'm a tech geek working in AI and always excited about the latest innovations. I love hiking, exploring and love nature and spirituality. Also i am open for all religion be it hindu, muslim, jews.

Your profile information:
Religion: christianity
Partner Preferences: i am looking for a hindu girl who outdoor activities and cooking and a software engineer
Full Profile: Hey there! I'm a tech geek working in AI and always excited about the latest innovations. I love hiking, exploring and love nature and spirituality. Also i am open for all religion be it hindu, muslim, jews.

Top matches for you:
1. Match (Score: 0.29):
   Christian seeking same. I'm

This model rejected all the profiles that had other religions mentioned in their bio becasue the user was a Christian. It used NLP and transformers to understand the symentic meaning for exactly what the user is lookign for in a person and creates a most relavent suggestions score.

